## Combining multiecho sequences 

This workflow is to be run on the T2 data after preprocessing with fmriprep.
Using the equation detailed in https://www.sciencedirect.com/science/article/pii/S1053811917310194
to combine the echoes.

In [ ]:
from bids.layout import BIDSLayout
from nipype.interfaces import afni 
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
from nipype.interfaces.utility import Function
import nipype.interfaces.fsl.maths as fsl
import pandas as pd
import os, re, json

As is tradition, I'll first set out the variables that will apply across the workflow

In [ ]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
layout = BIDSLayout(Basedir)
subs = layout.get_subjects()
sess = layout.get_sessions()

combi = pe.Workflow(name="combine-echoes") # workflow to run the analysis

Now I will define the nodes required for the workflow: datagrabber, datasink, combining images using fsl maths.

## Data grabber Node

In [ ]:
dgT2s = pe.Node(DataGrabber(infields=['sub', 'sess'], 
                            outfields=['func']), name='T2-grabber')
dgT2s.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dgT2s.inputs.sort_filelist = True
dgT2s.inputs.template='*'
dgT2s.inputs.template_args = {'func': [['sub', 'sess', 'sub']]}
dgT2s.inputs.field_template = {'func': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-%s/func/sub-%s_*-TR700_*_space-T1w_desc-preproc_bold.nii.gz'}

In [ ]:
# debugging
# dgT2s.inputs.sub = '01'
# dgT2s.inputs.sess = '02'
# res = dgT2s.run()
# res.outputs

In [ ]:
T2inf = pe.Node(ni.IdentityInterface(fields=['sub', 'sess']),
                   name='T2-info')
#T2inf.iterables = [('sub', layout.get_subjects()), ('sess', layout.get_session())]
T2inf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('sess', layout.get_session())]

In [ ]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and session folder strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    l = fname.split(os.sep)
    name = [[s for s in l if re.search('sub', s)][0], [s for s in l if re.search('ses', s)][0]]
    name = '/'.join(name)
    return name

# Combine echoes

Aim: weighted summation of the two images

Formula for weights:
w_n = \frac{AVG_n\cdot TE}\sum {AVG_n\cdot TE}

Steps:
1. for each echo, calculate the average (over the dimension of time)
2. multiply the temporal averages by the TEs (10 & 30.56)
3. sum the outputs
4. calculate the avg x TE over the sum
5. multiply the original images by the weights
6. sum the outputs

### Temporal average

In [ ]:
getmu = pe.MapNode(fsl.MeanImage(), name='mu-image', iterfield=['in_file'])
getmu.inputs.dimension = 'T'

### multiply by TEs

In [ ]:
wghtTE = pe.MapNode(fsl.BinaryMaths(), name='weight-TE', iterfield=['in_file', 'operand_value'])
wghtTE.inputs.operand_value = [10.00, 30.56]
wghtTE.inputs.operation = 'mul'

In [ ]:
wghtTE.outputs

### sum

sum {AVG_n\cdot TE}

In [ ]:
sumWghtTE = pe.Node(fsl.BinaryMaths(), name='sum-wghted')
sumWghtTE.inputs.operation = 'add'

In [ ]:
# just using a couple of fairly dumb functions to get the file that I want
def print_in_file(fname): 
    return fname[0]

def print_op_file(fname): 
    return fname[1]

### Divide 

\frac{AVG_n\cdot TE}\sum {AVG_n\cdot TE}

In [ ]:
RatWgts = pe.MapNode(fsl.BinaryMaths(), name='ratio-wghted-2-sum', iterfield=['in_file'])
RatWgts.inputs.operation = 'div'

### Multiply original images by new weights

In [ ]:
MulWgts = pe.MapNode(fsl.BinaryMaths(), name='apply-weights', iterfield=['in_file', 'operand_file'])
MulWgts.inputs.operation = 'mul'

### Sum outputs

In [ ]:
sumOut = pe.Node(fsl.BinaryMaths(), name='sum-outs')
sumOut.inputs.operation = 'add'

In [ ]:
def PrintOutput(results):
    # use this to get the filename of the last output
    import pandas as pd
    import re
    import os
    fname = os.path.split(results[0])[-1] 
    fname = fname.replace('_echo-1', '')
    return fname

# Datasink

In [ ]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/"

In [ ]:
substitutions = [('_sess_([0-9]*)_sub_([0-9]*)', '')]

In [ ]:
ds.inputs.regexp_substitutions = substitutions
ds.inputs

## Connect workflow

In [ ]:
combi.connect([(T2inf, dgT2s, [('sub',  'sub')]), 
               (T2inf, dgT2s, [('sess', 'sess')]),
               (dgT2s, ds, [(('func', printSubPath),
                              'container')]),
               (dgT2s, getmu, [('func', 'in_file')]),
               (getmu, wghtTE,[('out_file', 'in_file')]),
               (wghtTE, sumWghtTE, [(('out_file', print_in_file), 'in_file')]),
               (wghtTE, sumWghtTE, [(('out_file', print_op_file), 'operand_file')]),
               (wghtTE, RatWgts, [('out_file', 'in_file')]),
               (sumWghtTE, RatWgts, [('out_file', 'operand_file')]),
               (RatWgts, MulWgts, [('out_file', 'operand_file')]),
               (dgT2s, MulWgts, [('func', 'in_file')]),
               (MulWgts, sumOut, [(('out_file', print_in_file), 'in_file')]),
               (MulWgts, sumOut, [(('out_file', print_op_file), 'operand_file')]),
               (dgT2s, sumOut, [(('func', PrintOutput), 'out_file')]),
               (sumOut, ds, [("out_file", "func.@combd")])
              ])
combi.run()